# 1. Preparation

## 1.1 Import Libraries

In [5]:
import nltk
import pandas as pd
import string

## 1.2 Load csv files to data frames


In [6]:
#train_df = pd.read_csv('data/train.csv',encoding = "latin1")
#test_df = pd.read_csv('data/test.csv',encoding = "latin1")
#attributes_df = pd.read_csv('data/attributes.csv',encoding = "latin1")
#product_descriptions_df = pd.read_csv('data/product_descriptions.csv',encoding = "latin1")
#sample_submission_df = pd.read_csv('data/sample_submission.csv', encoding = "latin1")
from pyspark.sql import SparkSession
spark  = SparkSession.builder.appName("task14").getOrCreate()
train_df = spark.read.option("header", "true").option("mode", "DROPMALFORMED").csv("data/train.csv")
product_descriptions_df = spark.read.option("header", "true").option("mode", "DROPMALFORMED").csv("data/product_descriptions.csv")

In [ ]:
merged_df = train_df.join(product_descriptions_df,'product_uid')
merged_rdd = merged_df.rdd.map(list)

In [7]:
print(merged_rdd.first())

('100001', '3', 'Simpson Strong-Tie 12-Gauge Angle', 'l bracket', '2.5', '"Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws"')


In [26]:
#print(train_df.dtypes)

id                 int64
product_uid        int64
product_title     object
search_term       object
relevance        float64
dtype: object


In [28]:
#print(product_descriptions_df.dtypes)

product_uid             int64
product_description    object
dtype: object


In [29]:
#print(product_descriptions_df.head())

   product_uid                                product_description
0       100001  Not only do angles make joints stronger, they ...
1       100002  BEHR Premium Textured DECKOVER is an innovativ...
2       100003  Classic architecture meets contemporary design...
3       100004  The Grape Solar 265-Watt Polycrystalline PV So...
4       100005  Update your bathroom with the Delta Vero Singl...


# 2. Data Cleaning
## 2.1 Download NLTK's stopwords list and WordNetLemmatizer

In [8]:
lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()
stopwords=nltk.corpus.stopwords.words('english')
# nltk.download('stopwords')
# nltk.download('wordnet')


Process and tokenize the raw text by:
    1. Convert to lower case
    2. Remove apostrophe
    3. Remove Punctuation
    4. Lemmatize

In [9]:
def process(text, lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()):
   
    # convert text to lower case
    lowercase = str(text).lower()
    
    #remove 's from string
    apoRemoved = lowercase.replace("'s","")
    
    #convert don't to dont
    apoRemoved = apoRemoved.replace("'","")
    
    #handle other punctuations
    transtable = str.maketrans(string.punctuation,"                                ")
    brokenWords = apoRemoved.translate(transtable)
    
    #convert string to list of words
    listOfWords =  nltk.word_tokenize(brokenWords)
    
    #lemmatize text
    lemmatizedList=[lemmatizer.lemmatize(word) for word in listOfWords]
   
    return lemmatizedList


In [32]:
# test case
text = "Dogs Here's don't cars."
print(process(text))

['dog', 'here', 'dont', 'car']


In [33]:
# process train_df
def process_train_df(df, lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()):
    
    newdf=df
    for i,row in newdf.iterrows():
#         if i>5:
#             break
#         print("raw: ",text)
        newdf.at[i,'product_title'] = process(row['product_title'])
#         print("processed: ",df.iloc[i]['product_title'])
    return newdf


In [92]:
#processed_train_df = process_train_df(train_df)
#print(processed_train_df.head())

In [35]:
# process 
def process_product_descriptions_df(df, lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()):
    
    newdf=df
    for i,row in newdf.iterrows():
#         if i>5:
#             break
#         print("raw: ",text)
        newdf.at[i,'product_description'] = process(row['product_description'])
#         print("processed: ",df.iloc[i]['product_title'])
    return newdf

In [91]:
#processed_product_descriptions_df = process_product_descriptions_df(product_descriptions_df)
#print(processed_product_descriptions_df.head())

In [10]:
processed_rdd = merged_rdd.map(lambda row: [row[0], row[1], process(row[2]), process(row[3]), float(row[4]), process(row[5])])

In [11]:
processed_rdd.first()

['100001',
 '3',
 ['simpson', 'strong', 'tie', '12', 'gauge', 'angle'],
 ['l', 'bracket'],
 2.5,
 ['not',
  'only',
  'do',
  'angle',
  'make',
  'joint',
  'stronger',
  'they',
  'also',
  'provide',
  'more',
  'consistent',
  'straight',
  'corner',
  'simpson',
  'strong',
  'tie',
  'offer',
  'a',
  'wide',
  'variety',
  'of',
  'angle',
  'in',
  'various',
  'size',
  'and',
  'thickness',
  'to',
  'handle',
  'light',
  'duty',
  'job',
  'or',
  'project',
  'where',
  'a',
  'structural',
  'connection',
  'is',
  'needed',
  'some',
  'can',
  'be',
  'bent',
  'skewed',
  'to',
  'match',
  'the',
  'project',
  'for',
  'outdoor',
  'project',
  'or',
  'those',
  'where',
  'moisture',
  'is',
  'present',
  'use',
  'our',
  'zmax',
  'zinc',
  'coated',
  'connector',
  'which',
  'provide',
  'extra',
  'resistance',
  'against',
  'corrosion',
  'look',
  'for',
  'a',
  'z',
  'at',
  'the',
  'end',
  'of',
  'the',
  'model',
  'number',
  'versatile',
  'conn

# 3. Feature Engineering

## 3.1 Iteration 1: Calculating the tfidf similarities between description, title and search

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

def dummy_fun(doc):
    return doc
def tfidf_sim(row, stopwords=nltk.corpus.stopwords.words('english')):
    title = row[2]
    search = row[3]
    relevance = row[4]
    desc = row[5]
    title_search = []
    title_search.append(title)
    title_search.append(search)
    desc_search = []
    desc_search.append(desc)
    desc_search.append(search)
    vect = TfidfVectorizer(analyzer='word',tokenizer=dummy_fun,preprocessor=dummy_fun,token_pattern=None,stop_words=stopwords,use_idf=False)
    tfidf1 = vect.fit_transform(title_search)
    tfidf2 = vect.fit_transform(desc_search)
    title_sim_value = (tfidf1 * tfidf1.T).A[0,1]
    desc_sim_value = (tfidf2 * tfidf2.T).A[0,1]
    row.append(float(title_sim_value))
    row.append(float(desc_sim_value))
    return row

In [13]:
new_rdd = processed_rdd.map(tfidf_sim)

In [14]:
new_rdd.take(2)

[['100001',
  '3',
  ['simpson', 'strong', 'tie', '12', 'gauge', 'angle'],
  ['l', 'bracket'],
  2.5,
  ['not',
   'only',
   'do',
   'angle',
   'make',
   'joint',
   'stronger',
   'they',
   'also',
   'provide',
   'more',
   'consistent',
   'straight',
   'corner',
   'simpson',
   'strong',
   'tie',
   'offer',
   'a',
   'wide',
   'variety',
   'of',
   'angle',
   'in',
   'various',
   'size',
   'and',
   'thickness',
   'to',
   'handle',
   'light',
   'duty',
   'job',
   'or',
   'project',
   'where',
   'a',
   'structural',
   'connection',
   'is',
   'needed',
   'some',
   'can',
   'be',
   'bent',
   'skewed',
   'to',
   'match',
   'the',
   'project',
   'for',
   'outdoor',
   'project',
   'or',
   'those',
   'where',
   'moisture',
   'is',
   'present',
   'use',
   'our',
   'zmax',
   'zinc',
   'coated',
   'connector',
   'which',
   'provide',
   'extra',
   'resistance',
   'against',
   'corrosion',
   'look',
   'for',
   'a',
   'z',
   'at',

In [15]:
new_df = new_rdd.toDF(["product_id","id","title","search_term","label","descriptions","title_sim","desc_sim"])

In [16]:
#new_df.show()
new_df.select(new_df["title_sim"],new_df["desc_sim"]).show()

+-------------------+-------------------+
|          title_sim|           desc_sim|
+-------------------+-------------------+
|                0.0|                0.0|
| 0.2886751345948129|0.11826247919781652|
|                0.0|0.22808577638091165|
|0.42640143271122083|0.15339299776947407|
|0.17407765595569785|0.06262242910851495|
| 0.2886751345948129|0.13784910335911552|
| 0.2041241452319315|0.09747403576571587|
| 0.2041241452319315|0.09747403576571587|
| 0.4472135954999579|0.24077170617153837|
| 0.5773502691896258|0.27407548393101266|
|0.26726124191242434|0.36293309315564887|
| 0.3721042037676254| 0.1900028500641266|
|                0.0|                0.0|
|  0.282842712474619| 0.7009039702739965|
|                0.0|0.14213381090374028|
|                0.0|                0.0|
|                0.0|                0.0|
|                0.0|0.07106690545187014|
| 0.3481553119113957| 0.2057377999494559|
|0.17407765595569785| 0.3086066999241839|
+-------------------+-------------

In [17]:
# Features optimization 1 - "title_sim"
from pyspark.ml.feature import VectorAssembler
# delete desct_sim
assembler = VectorAssembler(inputCols=["title_sim"],outputCol="features")
final_df = assembler.transform(new_df).drop('product_id')

In [18]:
final_df.select("features").show(truncate=False)

+---------------------+
|features             |
+---------------------+
|[0.0]                |
|[0.2886751345948129] |
|[0.0]                |
|[0.42640143271122083]|
|[0.17407765595569785]|
|[0.2886751345948129] |
|[0.2041241452319315] |
|[0.2041241452319315] |
|[0.4472135954999579] |
|[0.5773502691896258] |
|[0.26726124191242434]|
|[0.3721042037676254] |
|[0.0]                |
|[0.282842712474619]  |
|[0.0]                |
|[0.0]                |
|[0.0]                |
|[0.0]                |
|[0.3481553119113957] |
|[0.17407765595569785]|
+---------------------+
only showing top 20 rows



In [22]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(maxIter=5)

In [20]:
lrModel = lr.fit(final_df)
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [0.0]
Intercept: 2.381798094242825
numIterations: 1
objectiveHistory: [0.4999999999999982]
+--------------------+
|           residuals|
+--------------------+
| 0.11820190575717504|
|   0.618201905757175|
|   0.618201905757175|
| 0.28820190575717497|
|-0.05179809424282489|
|   0.618201905757175|
| 0.28820190575717497|
|   0.618201905757175|
| 0.28820190575717497|
|   0.618201905757175|
| 0.28820190575717497|
| 0.28820190575717497|
| 0.28820190575717497|
|   0.618201905757175|
|-0.05179809424282489|
|-0.05179809424282489|
|  -0.711798094242825|
|  -1.381798094242825|
|-0.05179809424282489|
|-0.05179809424282489|
+--------------------+
only showing top 20 rows

RMSE: 0.534005
r2: -0.000000


In [ ]:
# Features optimization 2 - "desc_sim"

In [23]:
assembler = VectorAssembler(inputCols=["desc_sim"],outputCol="features")
final_df = assembler.transform(new_df).drop('product_id')
lrModel = lr.fit(final_df)
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [0.834621736855]
Intercept: 2.214378992500123
numIterations: 1
objectiveHistory: [0.0]
+--------------------+
|           residuals|
+--------------------+
| 0.28562100749987707|
|  0.6869165717070276|
|    0.59525566066494|
| 0.32759587728013706|
| 0.06335496695125498|
|     0.6705691494304|
| 0.37426705847083763|
|  0.7042670584708377|
| 0.25466770790946747|
|  0.5568716510720253|
|  0.1527091589281846|
|  0.2970404987719726|
|   0.455621007499877|
|   0.200631318461292|
|-0.00300696062241...|
| 0.11562100749987714|
|  -0.544378992500123|
| -1.2736929765612666|
|-0.05609223243064587|
|-0.14194885239590738|
+--------------------+
only showing top 20 rows

RMSE: 0.520745
r2: 0.049048


In [24]:
# Features optimization 3 - "desc_sim, title_sim"
assembler = VectorAssembler(inputCols=["title_sim","desc_sim"],outputCol="features")
final_df = assembler.transform(new_df).drop('product_id')
lrModel = lr.fit(final_df)
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [0.750607156948,0.383934451293]
Intercept: 2.0528862643397696
numIterations: 1
objectiveHistory: [0.0]
+--------------------+
|           residuals|
+--------------------+
|  0.4471137356602304|
|  0.6850270735411228|
|  0.8595437482576194|
| 0.23816091210357548|
|  0.1224068932767719|
|  0.6775070937410663|
|  0.4264730509062562|
|  0.7564730509062563|
|  0.1889914572977247|
| 0.40852347100481445|
| 0.27716301672913923|
| 0.26486101719839006|
|  0.6171137356602303|
|  0.4657087901498387|
| 0.22254366896067745|
| 0.27711373566023045|
| -0.3828862643397697|
|  -1.080171297689546|
|-0.06320396252370841|
|0.027965057234376367|
+--------------------+
only showing top 20 rows

RMSE: 0.507310
r2: 0.097482


In [25]:
# Try word2vec
#train_rdd = train_df.rdd.map(tuple)
#product_desc_rdd = product_descriptions_df.rdd.map(tuple)
#title_rdd = train_rdd.map(lambda row: process(row[2]))
#desc_rdd = product_desc_rdd.map(lambda row: process(row[1]))

In [26]:
#print(title_rdd.first())
#print(desc_rdd.first())

In [27]:
#corpus_rdd = title_rdd.union(desc_rdd)

In [28]:
#corpus_rdd.saveAsTextFile("corpus")

In [29]:
#from pyspark.sql import Row
#corpus_df = corpus_rdd.map(lambda x: Row(x)).toDF(["corpus"])

## 3.2 Iteration 2: Adding features like diceCoefficent, overlap and search length

In [ ]:
# Adding diceCoefficient & overlap & search length

In [46]:
def diceCoefficient(row):
    title_set = set(row[2])
    search_set = set(row[3])
    desc_set = set(row[5])
    overlap_title = len(title_set & search_set)
    dice_title = float(overlap_title * 2.0/(len(title_set) + len(search_set)))
    overlap_desc = len(desc_set & search_set)
    dice_desc = float(overlap_desc * 2.0/(len(desc_set) + len(search_set)))
    row.append(dice_title)
    row.append(dice_desc)
    row.append(len(row[3]))
    row.append(overlap_title)
    row.append(overlap_desc)
    return row

In [47]:
new_rdd2 = new_rdd.map(diceCoefficient)

In [48]:
new_rdd2.take(2)

[['100001',
  '3',
  ['simpson', 'strong', 'tie', '12', 'gauge', 'angle'],
  ['l', 'bracket'],
  2.5,
  ['not',
   'only',
   'do',
   'angle',
   'make',
   'joint',
   'stronger',
   'they',
   'also',
   'provide',
   'more',
   'consistent',
   'straight',
   'corner',
   'simpson',
   'strong',
   'tie',
   'offer',
   'a',
   'wide',
   'variety',
   'of',
   'angle',
   'in',
   'various',
   'size',
   'and',
   'thickness',
   'to',
   'handle',
   'light',
   'duty',
   'job',
   'or',
   'project',
   'where',
   'a',
   'structural',
   'connection',
   'is',
   'needed',
   'some',
   'can',
   'be',
   'bent',
   'skewed',
   'to',
   'match',
   'the',
   'project',
   'for',
   'outdoor',
   'project',
   'or',
   'those',
   'where',
   'moisture',
   'is',
   'present',
   'use',
   'our',
   'zmax',
   'zinc',
   'coated',
   'connector',
   'which',
   'provide',
   'extra',
   'resistance',
   'against',
   'corrosion',
   'look',
   'for',
   'a',
   'z',
   'at',

In [49]:
new_df2 = new_rdd2.toDF(["product_id","id","title","search_term","label","descriptions","title_sim","desc_sim","dice_title","dice_desc","search_length","overlap_title","overlap_desc"])

In [50]:
new_df2.select(new_df2["title_sim"],new_df2["desc_sim"],new_df2["dice_title"],new_df2["dice_desc"],new_df2["search_length"],new_df2["overlap_title"],new_df2["overlap_desc"]).show()

+-------------------+-------------------+-------------------+--------------------+-------------+-------------+------------+
|          title_sim|           desc_sim|         dice_title|           dice_desc|search_length|overlap_title|overlap_desc|
+-------------------+-------------------+-------------------+--------------------+-------------+-------------+------------+
|                0.0|                0.0|                0.0|                 0.0|            2|            0|           0|
| 0.2886751345948129|0.11826247919781652|               0.25|  0.0196078431372549|            2|            1|           1|
|                0.0|0.22808577638091165|                0.0|0.016129032258064516|            2|            0|           1|
|0.42640143271122083|0.15339299776947407|0.35294117647058826| 0.06896551724137931|            3|            3|           3|
|0.17407765595569785|0.06262242910851495|0.11764705882352941|0.022988505747126436|            3|            1|           1|
| 0.2886

In [41]:
# Features optimization 4 - "desc_sim, title_sim, dice_title"
assembler2 = VectorAssembler(inputCols=["title_sim","desc_sim","dice_title"],outputCol="features")
final_df2 = assembler2.transform(new_df2)

In [42]:
lrModel = lr.fit(final_df2)
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [1.25672083615,0.386217257614,-0.629575113796]
Intercept: 2.0508369390315
numIterations: 1
objectiveHistory: [0.0]
+--------------------+
|           residuals|
+--------------------+
|     0.4491630609685|
|  0.6980977725001596|
|  0.8610723979139037|
|  0.2462554543212896|
| 0.11027784123244988|
|  0.6072069622281968|
| 0.45753196893299597|
|  0.7249426301345201|
|  0.1740086004920478|
|  0.3695723116579366|
| 0.28302484647016257|
| 0.26702210932390447|
|  0.6191630609684999|
|  0.5167229412288039|
|  0.2242685303070302|
|  0.2791630609685001|
|-0.38083693903150007|
| -1.0782842043622347|
|-0.06994376559767757|
|0.025150158801394262|
+--------------------+
only showing top 20 rows

RMSE: 0.506218
r2: 0.101363


In [43]:
# Features optimization 5 - "desc_sim, title_sim, dice_title, dice_desc"
assembler2 = VectorAssembler(inputCols=["title_sim","desc_sim","dice_title", "dice_desc"],outputCol="features")
final_df2 = assembler2.transform(new_df2)

In [44]:
lrModel = lr.fit(final_df2)
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [1.15134844323,0.54986352137,-0.413602170541,-1.38904425294]
Intercept: 2.059537263139896
numIterations: 1
objectiveHistory: [0.0]
+--------------------+
|           residuals|
+--------------------+
|  0.4404627368601042|
|  0.6737055512459604|
|  0.8374506282484608|
|  0.2769542892479602|
| 0.11619603996746619|
|  0.5907745227010874|
| 0.42841680257650605|
|  0.7075851019820618|
|  0.1566016419874856|
|  0.4167449110001602|
| 0.23315585380849235|
| 0.23952918116991606|
|  0.6104627368601041|
| 0.41706612964631384|
|  0.2897853287708667|
| 0.27046273686010425|
| -0.3895372631398959|
| -1.0498759671845144|
|-0.05151065305807023|
| 0.07805681620898586|
+--------------------+
only showing top 20 rows

RMSE: 0.505061
r2: 0.105466


In [45]:
# Features optimization 6 - "desc_sim, title_sim, dice_title, dice_desc, length"
assembler2 = VectorAssembler(inputCols=["title_sim","desc_sim","dice_title","dice_desc","search_length"],outputCol="features")
final_df2 = assembler2.transform(new_df2)
lrModel = lr.fit(final_df2)
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [0.812656456634,0.453785887724,0.123381519568,-0.353887523512,-0.0704905741084]
Intercept: 2.229513247931489
numIterations: 1
objectiveHistory: [0.0]
+-------------------+
|          residuals|
+-------------------+
|0.41146790028531255|
| 0.5993019353370994|
| 0.8136736570554226|
|  0.216692636752581|
| 0.1356958412188427|
| 0.5318151354460925|
|0.42083511365014736|
| 0.6901358617863695|
| 0.0818064097348623|
|  0.371218448837487|
|0.18185984370471742|
| 0.4474537230434925|
| 0.5109773261769117|
| 0.3409764830038138|
|0.20180379490496936|
|0.24146790028531262|
| -0.277550951497886|
|-1.1083641524048593|
|-0.0764179233348008|
|0.04522627848866945|
+-------------------+
only showing top 20 rows

RMSE: 0.498900
r2: 0.127157


In [52]:
# Features optimization 7 - "desc_sim, title_sim, dice_title, dice_desc, length, overlap"
assembler2 = VectorAssembler(inputCols=["title_sim","desc_sim","dice_title","dice_desc","search_length","overlap_title","overlap_desc"],outputCol="features")
final_df2 = assembler2.transform(new_df2)
lrModel = lr.fit(final_df2)
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [0.550540148443,0.260441596816,0.0798898998219,-1.4094141844,-0.115189328945,0.0511733971402,0.0800311166064]
Intercept: 2.3077846240688498
numIterations: 1
objectiveHistory: [0.0]
+--------------------+
|           residuals|
+--------------------+
|  0.4225940338206682|
|  0.6093248969842651|
|  0.8058923802478146|
| 0.10847333918790891|
| 0.14743413936913985|
|  0.4819330116066922|
| 0.31057182996497934|
|  0.6546734858013208|
| 0.05101560022024065|
|  0.3511072642282822|
| 0.24058937813194614|
|  0.4198664578297975|
| 0.47740470487590914|
| 0.35862155812791174|
|  0.1544205024888612|
| 0.25259403382066825|
| -0.1770273082898144|
| -1.1264927318452354|
|-0.07827907952960311|
| 0.01401394399024003|
+--------------------+
only showing top 20 rows

RMSE: 0.495088
r2: 0.140445


## 3.3 Iteration 3: Adding feature ratio - the ratio bewteen description length, title length and search length

In [ ]:
# Features optimization 8 - "desc_sim, title_sim, dice_title, dice_desc, length, overlap, ratio"

In [56]:
def ratio(row):
    title_set = set(row[2])
    search_set = set(row[3])
    desc_set = set(row[5])
    ratio_title_search = float(len(search_set)/len(title_set))
    ratio_desc_search = float(len(search_set)/len(desc_set))
    row.append(ratio_title_search)
    row.append(ratio_desc_search)
    return row

In [58]:
new_rdd3 = new_rdd2.map(ratio)
new_rdd3.take(2)

[['100001',
  '3',
  ['simpson', 'strong', 'tie', '12', 'gauge', 'angle'],
  ['l', 'bracket'],
  2.5,
  ['not',
   'only',
   'do',
   'angle',
   'make',
   'joint',
   'stronger',
   'they',
   'also',
   'provide',
   'more',
   'consistent',
   'straight',
   'corner',
   'simpson',
   'strong',
   'tie',
   'offer',
   'a',
   'wide',
   'variety',
   'of',
   'angle',
   'in',
   'various',
   'size',
   'and',
   'thickness',
   'to',
   'handle',
   'light',
   'duty',
   'job',
   'or',
   'project',
   'where',
   'a',
   'structural',
   'connection',
   'is',
   'needed',
   'some',
   'can',
   'be',
   'bent',
   'skewed',
   'to',
   'match',
   'the',
   'project',
   'for',
   'outdoor',
   'project',
   'or',
   'those',
   'where',
   'moisture',
   'is',
   'present',
   'use',
   'our',
   'zmax',
   'zinc',
   'coated',
   'connector',
   'which',
   'provide',
   'extra',
   'resistance',
   'against',
   'corrosion',
   'look',
   'for',
   'a',
   'z',
   'at',

In [59]:
new_df3 = new_rdd3.toDF(["product_id","id","title","search_term","label","descriptions","title_sim","desc_sim","dice_title","dice_desc","search_length","overlap_title","overlap_desc","ratio_title_search","ratio_desc_search"])

In [61]:
assembler3 = VectorAssembler(inputCols=["title_sim","desc_sim","dice_title","dice_desc","search_length","overlap_title","overlap_desc","ratio_title_search","ratio_desc_search"],outputCol="features")
final_df3 = assembler3.transform(new_df3)

In [62]:
lrModel = lr.fit(final_df3)
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [0.526773376902,0.255081305005,0.168739826551,-0.522897730475,-0.0945299839446,0.0426523930343,0.0581759836408,-0.0584987115373,-1.10164518371]
Intercept: 2.3076288760079744
numIterations: 1
objectiveHistory: [0.0]
+--------------------+
|           residuals|
+--------------------+
| 0.42296356606794916|
|  0.6079702064261125|
|    0.80056808399558|
| 0.10811792570557399|
|  0.1415082157557941|
| 0.49024998530183206|
|  0.3064162324693531|
|  0.6482706797862314|
| 0.05162935906044552|
|  0.3465558617758222|
| 0.24181356352049743|
|    0.40218215845145|
| 0.48331782383773625|
|  0.3579246990835747|
| 0.14727759137588237|
| 0.26319065905101313|
|-0.15598980588999978|
| -1.1247658747865525|
|-0.08034325020889854|
|0.007470689159555732|
+--------------------+
only showing top 20 rows

RMSE: 0.494955
r2: 0.140908


## 3.4 Iteration 4: Adding feature match - the number of of words matched between the title and search term.  
### However the result get worse in this iteration

In [63]:
# Ratio has little improvement
# Try the number of words matched between the title and search term
# Features optimization 9 - "desc_sim, title_sim, dice_title, dice_desc, length, overlap, ratio, matchwords"

In [64]:
def matchWords(row):
    title = row[2]
    term = row[3]
    l1=len(title)
    l2=len(term)
    match=0
    for i in range(l1):
        for j in range(l2):
            if title[i] == term[j]:
                match+=2
            elif title[i] in term[j]:
                match+=1
            elif term[j] in title[i]:
                match+=1
    row.append(match)
    return row

In [68]:
new_rdd4 = new_rdd3.map(ratio)
new_df4 = new_rdd4.toDF(["product_id","id","title","search_term","label","descriptions","title_sim","desc_sim","dice_title","dice_desc","search_length","overlap_title","overlap_desc","ratio_title_search","ratio_desc_search","match"])
assembler4 = VectorAssembler(inputCols=["title_sim","desc_sim","dice_title","dice_desc","search_length","overlap_title","overlap_desc","ratio_title_search","ratio_desc_search","match"],outputCol="features")
final_df4 = assembler4.transform(new_df4)


In [69]:
lrModel = lr.fit(final_df4)
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [0.456174551297,0.328451478469,0.425133709743,0.379866901568,-0.0580376001609,0.0196120165581,0.0214073840427,-0.0942935365018,-2.29854298325,-0.0942935365018]
Intercept: 2.2930886617296995
numIterations: 6
objectiveHistory: [0.4999999999999982, 0.4492671761852307, 0.4338728554955402, 0.4318184650475825, 0.4308402805724434, 0.43017949399721545]
+--------------------+
|           residuals|
+--------------------+
| 0.43181975592487243|
|  0.6065388211352212|
|  0.7872315869291966|
| 0.12932772525749492|
| 0.13378054559584207|
|  0.5142037974371534|
| 0.30883296823286166|
|   0.646798315997164|
|0.055578995695534594|
|  0.3565872964922159|
| 0.22459169028548587|
| 0.37729559455644557|
| 0.49957098137251554|
| 0.31975011185713464|
| 0.15813117955882028|
|  0.2742873344017718|
| -0.1483366694667574|
| -1.1137907430197043|
|-0.07988987385569857|
|0.009449051674842668|
+--------------------+
only showing top 20 rows

RMSE: 0.495319
r2: 0.139641


## 3.5 Iteration 5: 2-gram tfidf. 
### However, compared to 1-gram tfidf, the performance get slightly worse

In [86]:
def dummy_fun(doc):
    return doc
def tfidf2_sim(row, stopwords=nltk.corpus.stopwords.words('english')):
    title = row[2]
    search = row[3]
    relevance = row[4]
    desc = row[5]
    title_search = []
    title_search.append(title)
    title_search.append(search)
    desc_search = []
    desc_search.append(desc)
    desc_search.append(search)
    vect = TfidfVectorizer(analyzer='word',tokenizer=dummy_fun,preprocessor=dummy_fun,ngram_range=(1,2),token_pattern=None,stop_words=stopwords,use_idf=False)
    tfidf1 = vect.fit_transform(title_search)
    tfidf2 = vect.fit_transform(desc_search)
    title_sim_value = (tfidf1 * tfidf1.T).A[0,1]
    desc_sim_value = (tfidf2 * tfidf2.T).A[0,1]
    row.append(float(title_sim_value))
    row.append(float(desc_sim_value))
    return row

In [87]:
new_rdd5 = processed_rdd.map(tfidf2_sim)

In [88]:
new_rdd5.take(2)

[['100001',
  '3',
  ['simpson', 'strong', 'tie', '12', 'gauge', 'angle'],
  ['l', 'bracket'],
  2.5,
  ['not',
   'only',
   'do',
   'angle',
   'make',
   'joint',
   'stronger',
   'they',
   'also',
   'provide',
   'more',
   'consistent',
   'straight',
   'corner',
   'simpson',
   'strong',
   'tie',
   'offer',
   'a',
   'wide',
   'variety',
   'of',
   'angle',
   'in',
   'various',
   'size',
   'and',
   'thickness',
   'to',
   'handle',
   'light',
   'duty',
   'job',
   'or',
   'project',
   'where',
   'a',
   'structural',
   'connection',
   'is',
   'needed',
   'some',
   'can',
   'be',
   'bent',
   'skewed',
   'to',
   'match',
   'the',
   'project',
   'for',
   'outdoor',
   'project',
   'or',
   'those',
   'where',
   'moisture',
   'is',
   'present',
   'use',
   'our',
   'zmax',
   'zinc',
   'coated',
   'connector',
   'which',
   'provide',
   'extra',
   'resistance',
   'against',
   'corrosion',
   'look',
   'for',
   'a',
   'z',
   'at',

In [89]:
new_df5 = new_rdd5.toDF(["product_id","id","title","search_term","label","descriptions","title_sim","desc_sim"])

In [90]:
# Features optimization 10 - "desc_sim, title_sim"
assembler5 = VectorAssembler(inputCols=["title_sim","desc_sim"],outputCol="features")
final_df5 = assembler5.transform(new_df5).drop('product_id')
lrModel = lr.fit(final_df5)
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [0.879461665527,0.521748834226]
Intercept: 2.1020815080812127
numIterations: 1
objectiveHistory: [0.0]
+--------------------+
|           residuals|
+--------------------+
| 0.39791849191878725|
|  0.7061758048864828|
|  0.8076991833699525|
| 0.16339047519039962|
| 0.12346980712789657|
|  0.6547139558667938|
| 0.42750428756103087|
|  0.7575042875610309|
|  0.2570031362719525|
|  0.4159335946647116|
|  0.3026238501592551|
|  0.2732091712804925|
|  0.5679184919187872|
|  0.3393692825570973|
| 0.15881117059948124|
| 0.22791849191878732|
| -0.4320815080812128|
|  -1.125117281854315|
|-0.00798812086801...|
|-0.00247804061145...|
+--------------------+
only showing top 20 rows

RMSE: 0.507833
r2: 0.095621


## 3.6 Iteration 6: Calculating the word2vec similarities
### Worse than tfidf
### When combining with other features, it shows improvement

In [94]:
# public corpus : Wikipedia+Gigaword 5
# referencehttps://github.com/3Top/word2vec-api

In [95]:
from gensim.models import KeyedVectors
#filename = 'data/GoogleNews-vectors-negative300.bin'
#model = KeyedVectors.load_word2vec_format(filename, binary=True)
#model = KeyedVectors.load_word2vec_format(filename, binary=False)

In [96]:
filename = 'corpus/test_w2c.txt'
model = KeyedVectors.load_word2vec_format(filename, binary=False)

In [98]:
def remove_number(row):
    row2 = []
    row3 = []
    row5 = []
    for word in row[2]:
        if not word.isdigit():
            row2.append(word)
    for word in row[3]:
        if not word.isdigit():
            row3.append(word)
    for word in row[5]:
        if not word.isdigit():
            row5.append(word)
    return [row[0], row[1], row2, row3, row[4], row5]

In [99]:
new_processed_rdd = processed_rdd.map(remove_number)

In [100]:
new_processed_rdd.first()

['100001',
 '3',
 ['simpson', 'strong', 'tie', 'gauge', 'angle'],
 ['l', 'bracket'],
 2.5,
 ['not',
  'only',
  'do',
  'angle',
  'make',
  'joint',
  'stronger',
  'they',
  'also',
  'provide',
  'more',
  'consistent',
  'straight',
  'corner',
  'simpson',
  'strong',
  'tie',
  'offer',
  'a',
  'wide',
  'variety',
  'of',
  'angle',
  'in',
  'various',
  'size',
  'and',
  'thickness',
  'to',
  'handle',
  'light',
  'duty',
  'job',
  'or',
  'project',
  'where',
  'a',
  'structural',
  'connection',
  'is',
  'needed',
  'some',
  'can',
  'be',
  'bent',
  'skewed',
  'to',
  'match',
  'the',
  'project',
  'for',
  'outdoor',
  'project',
  'or',
  'those',
  'where',
  'moisture',
  'is',
  'present',
  'use',
  'our',
  'zmax',
  'zinc',
  'coated',
  'connector',
  'which',
  'provide',
  'extra',
  'resistance',
  'against',
  'corrosion',
  'look',
  'for',
  'a',
  'z',
  'at',
  'the',
  'end',
  'of',
  'the',
  'model',
  'number',
  'versatile',
  'connector'

In [108]:
def word2vec_sim(row, i):
    try:
        sim = model.n_similarity(row[i], row[3])
    
        row.append(float(sim))
        return row
    except:
        new_list1 = []
        new_list2 = []
        for w1 in row[i]:
            if w1 in model.vocab:
                new_list1.append(w1)
        for w2 in row[3]:
            if w2 in model.vocab:
                new_list2.append(w2)
        if len(new_list1)>0 and len(new_list2)>0:
            sim = model.n_similarity(new_list1, new_list2)
        else:
            sim = 0.0
        row.append(float(sim))
        return row

In [111]:
word2vec_rdd = new_processed_rdd.map(lambda row: word2vec_sim(row, 2))
word2vec_rdd = word2vec_rdd.map(lambda row: word2vec_sim(row, 5))

In [112]:
word2vec_rdd.first()

['100001',
 '3',
 ['simpson', 'strong', 'tie', 'gauge', 'angle'],
 ['l', 'bracket'],
 2.5,
 ['not',
  'only',
  'do',
  'angle',
  'make',
  'joint',
  'stronger',
  'they',
  'also',
  'provide',
  'more',
  'consistent',
  'straight',
  'corner',
  'simpson',
  'strong',
  'tie',
  'offer',
  'a',
  'wide',
  'variety',
  'of',
  'angle',
  'in',
  'various',
  'size',
  'and',
  'thickness',
  'to',
  'handle',
  'light',
  'duty',
  'job',
  'or',
  'project',
  'where',
  'a',
  'structural',
  'connection',
  'is',
  'needed',
  'some',
  'can',
  'be',
  'bent',
  'skewed',
  'to',
  'match',
  'the',
  'project',
  'for',
  'outdoor',
  'project',
  'or',
  'those',
  'where',
  'moisture',
  'is',
  'present',
  'use',
  'our',
  'zmax',
  'zinc',
  'coated',
  'connector',
  'which',
  'provide',
  'extra',
  'resistance',
  'against',
  'corrosion',
  'look',
  'for',
  'a',
  'z',
  'at',
  'the',
  'end',
  'of',
  'the',
  'model',
  'number',
  'versatile',
  'connector'

In [113]:
new_df6 = word2vec_rdd.toDF(["product_id","id","title","search_term","label","descriptions","title_w2v","desc_w2v"])

In [115]:
#new_df.show()
new_df6.select(new_df6["title_w2v"],new_df6["desc_w2v"]).show()

+--------------------+--------------------+
|           title_w2v|            desc_w2v|
+--------------------+--------------------+
|  0.4940440902407819|  0.4242058993429343|
|  0.7191197759587993|  0.4533282018238042|
|  0.6278471914042068|  0.8170047875604547|
|  0.8335654528909927|  0.6901031486375008|
|  0.7594438207653402|   0.724009884890052|
|  0.5989543186814787| 0.41211387983193287|
|  0.8010003836133329|  0.7044757497579195|
| 0.30381848433962566| 0.02244382637768967|
|  0.8627779894166276|  0.8293984926300915|
|  0.1647928661068554|-0.10027809748611155|
|  0.6043361002621344|  0.5387670111845435|
|   0.801857116166376|  0.7770434443389089|
|-0.07211062826322981|-0.27007137082584576|
|  0.4627432380244008|  0.3514889856124315|
|  0.6057814515400581|  0.7102125565220965|
| 0.29681955766817525|   0.256095430488245|
|  0.7745318525597138|  0.7949061057271175|
|   0.549952784139019|  0.6849802220215219|
|  0.5825366324923527| 0.12575200736365866|
|  0.7082144150244697|  0.575345

In [116]:
# Features optimization 11 - "desc_w2v, title_w2v"
assembler6 = VectorAssembler(inputCols=["title_w2v","desc_w2v"],outputCol="features")
final_df6 = assembler6.transform(new_df6).drop('product_id')
lrModel = lr.fit(final_df6)
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [0.785418538863,-0.626783595624]
Intercept: 2.2195337126550823
numIterations: 1
objectiveHistory: [0.0]
+--------------------+
|           residuals|
+--------------------+
| 0.15832019872880299|
|  0.4997949639802748|
|    0.79942866203156|
| 0.22831386014243105|
|-0.03221744979103258|
|   0.568342680927628|
| 0.26289957988119594|
|  0.5559090394917523|
| 0.29267782894927397|
|  0.5881822487275192|
|  0.3134998349682441|
|  0.3077109267867755|
|  0.3378270067507301|
|  0.6373266996911178|
|  0.0798238846397119|
|0.037855118799289045|
| -0.6596312814649141|
| -1.2221424583277196|
|-0.26824948804936044|
|-0.08516117725789707|
+--------------------+
only showing top 20 rows

RMSE: 0.525588
r2: 0.031278


In [ ]:
# Combine the previous features 
# Shows improvement  

In [117]:
new_rdd6 = new_rdd4.map(lambda row: word2vec_sim(row, 2))
new_rdd6 = new_rdd6.map(lambda row: word2vec_sim(row, 5))
new_df6 = new_rdd6.toDF(["product_id","id","title","search_term","label","descriptions","title_sim","desc_sim","dice_title","dice_desc","search_length","overlap_title","overlap_desc","ratio_title_search","ratio_desc_search","match","title_w2v","desc_w2v"])
assembler6 = VectorAssembler(inputCols=["title_sim","desc_sim","dice_title","dice_desc","search_length","overlap_title","overlap_desc","ratio_title_search","ratio_desc_search","title_w2v","desc_w2v"],outputCol="features")
final_df6 = assembler6.transform(new_df6)

In [118]:
lrModel = lr.fit(final_df6)
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [0.593319850873,0.277495371734,0.251692397653,-0.622795970102,-0.0774303613095,0.0322700043512,0.0607410517511,-0.0890652171676,-0.507591135128,-0.455166914901,-0.190871685418]
Intercept: 2.352488581444818
numIterations: 1
objectiveHistory: [0.0]
+--------------------+
|           residuals|
+--------------------+
|  0.4539717320972958|
|   0.633229635825141|
|  0.8479384660470242|
| 0.13752503399400684|
| 0.17777791313800817|
|  0.5128691942245447|
| 0.34591882642964267|
|  0.6027306617597228|
|  0.0979627830201788|
|  0.3731850050260106|
|  0.2502236200453778|
| 0.38742043080008237|
|  0.4096844240170374|
| 0.31222893631289406|
| 0.17419718739522283|
| 0.22765944352174694|
|-0.12656776845921414|
| -1.0989479482659115|
|-0.10895447035882366|
|  0.0278327370722522|
+--------------------+
only showing top 20 rows

RMSE: 0.493883
r2: 0.144624


# 4 Model Selection